In [175]:
### モジュールのインポート
import pandas as pd
import numpy as np

In [176]:
### 入出力データセットの名称
## 入力データ
gs_file = 'gender_submission.csv'
test_file = 'test.csv'
train_file = 'train.csv'

##出力データ
train_x_file = 'train_x.csv'
train_y_file = 'train_y.csv'
test_x_file = 'test_x.csv'

In [177]:
### データのインポート
df_gs = pd.read_csv('original_data/' + gs_file)
df_test = pd.read_csv('original_data/' + test_file)
df_train = pd.read_csv('original_data/' + train_file)

In [178]:
### 学習データをxとyに分割
train_y = df_train['Survived']
train_x = df_train.drop(columns=['Survived'])

In [179]:
### 学習データとテストデータの結合
all_x = pd.concat([train_x, df_test])

各カラムを加工する。
カテゴリ変数化は最後にまとめて行う。

In [180]:
### PassengerID
## 不要なので削除する
all_x = all_x.drop(columns=['PassengerId'])

In [181]:
### Survived
pass

In [182]:
### Pclass
pass

In [183]:
### Name
## 1. 苗字を抽出する
# 最初のカンマまでの文字列を抽出する関数を定義する
def extract_before_comma(name):
    comma_index = name.find(",")
    return name[:comma_index].strip() if comma_index != -1 else None
# 新しいカラム 'FamilyName' に抽出結果を入れる
all_x['FamilyName'] = all_x['Name'].apply(extract_before_comma)

## 2. 同じ苗字の出現回数をカウントする
all_x['SameFamilyName_Count'] = all_x.groupby('FamilyName')['FamilyName'].transform('count')

## 3. Name列とFamilyName列を削除する
all_x = all_x.drop(['Name', 'FamilyName'], axis=1)

In [184]:
### Sex
pass

In [185]:
### Age
pass

In [186]:
### SibSp
pass

In [187]:
### Parch
pass

In [188]:
### Ticket
## 1. 同じチケットの出現回数をカウントする
all_x['SameTicket_Count'] = all_x.groupby('Ticket')['Ticket'].transform('count')

## 2. Ticket列を削除する
all_x = all_x.drop(columns=['Ticket'])

In [189]:
### Fare
pass

In [190]:
### Cabin
## 1. カラムの空白の有無をフラグ化
all_x['Cabin_flag'] = all_x['Cabin'].apply(lambda x: 0 if pd.isna(x) or x == '' else 1)

## 2. Cabin列を削除する
all_x = all_x.drop(columns=['Cabin'])

In [191]:
### Embarked
all_x['Embarked'] = all_x['Embarked'].apply(lambda x: 'N' if pd.isna(x) or x == '' else x)

In [192]:
### カテゴリ変数化
## 対象：Sex, Cabin_flag, Embarked
## 方法：get_dummies関数

all_x = pd.get_dummies(all_x, columns=['Sex', 'Cabin_flag', 'Embarked'])

In [193]:
### 学習データとテストデータに再分割（参考：Kaggleで勝つ P138）
processed_train_x = all_x.iloc[:train_x.shape[0], :].reset_index(drop=True)
processed_test_x = all_x.iloc[train_x.shape[0]:, :].reset_index(drop=True)

In [ ]:
### データをcsvで保存
processed_train_x.to_csv('processed_data/' + train_x_file, index=False)
train_y.to_csv('processed_data/' + train_y_file, index=False)
processed_test_x.to_csv('processed_data/' + test_x_file, index=False)